<a href="https://colab.research.google.com/github/KatiaBravo/ai-counselor/blob/web-scraping/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [908 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ge

In [2]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import logging
logging.basicConfig(level=logging.DEBUG)


options = Options()

options = webdriver.ChromeOptions()
options.add_argument('--headless') # this is must
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('window-size=1920x1080')

chromedriver_autoinstaller.install()
service = Service('/usr/bin/chromedriver')

driver = webdriver.Chrome(options=options)

In [3]:
#works
try:
    # Replace with the URL of the page you want to scrape
    driver.get('https://assist.org/transfer/results?year=74&institution=113&agreement=79&agreementType=to&view=agreement&viewBy=major&viewSendingAgreements=false&viewByKey=74%2F113%2Fto%2F79%2FMajor%2F9df99d8d-2540-4677-891f-a559d4801840')
    print("Page Title:", driver.title)

    # Explicit wait until the specific element is present
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.rowSending')))

    # Locate all containers of interest
    containers = driver.find_elements(By.CSS_SELECTOR, 'div.articRow, h2.areaTitle, div.sectionAdvisements, div.rowSending')

    for container in containers:
        try:
            if container.tag_name == 'div' and 'articRow' in container.get_attribute('class'):
                # Extract the course prefix and title using CSS Selectors
                prefix = container.find_element(By.CSS_SELECTOR, '.prefixCourseNumber').text
                title = container.find_element(By.CSS_SELECTOR, '.courseTitle').text
            elif container.tag_name == 'h2' and 'areaTitle' in container.get_attribute('class'):
                # Print the area title header
                header_text = container.text
                print(f'Header: {header_text}')
            elif container.tag_name == 'div' and 'sectionAdvisements' in container.get_attribute('class'):
                # Print the section advisements
                advisement_text = container.text
                print(f'Advisement: {advisement_text}')
            elif container.tag_name == 'div' and 'rowSending' in container.get_attribute('class'):
                # Extract course information, conjunctions, and attributes from the right column
                course_lines = container.find_elements(By.CSS_SELECTOR, '.courseLine')
                for line in course_lines:
                    try:
                        prefix = line.find_element(By.CSS_SELECTOR, '.prefixCourseNumber').text
                        title = line.find_element(By.CSS_SELECTOR, '.courseTitle').text
                        units = line.find_element(By.CSS_SELECTOR, '.courseUnits').text
                        print(f'Course Prefix: {prefix}, Course Title: {title}, Units: {units}')
                    except Exception as e:
                        print("Error extracting data from course line:", e)

                # Print the conjunctions (e.g., 'And', 'Or')
                conjunctions = container.find_elements(By.CSS_SELECTOR, 'awc-view-conjunction div.conjunction')
                for conjunction in conjunctions:
                    print(f'Conjunction: {conjunction.text}')

                # Print any additional advisement or attribute information
                attributes = container.find_elements(By.CSS_SELECTOR, '.attributeContainer')
                for attribute in attributes:
                    print(f'Attribute: {attribute.text}')
        except Exception as e:
            print("Error extracting data from container:", e)

except Exception as e:
    print("An error occurred:", e)
finally:
    driver.quit()

Page Title: Welcome to ASSIST
Header: COLLEGE OF ENGINEERING JUNIOR TRANSFER ADMISSION REQUIREMENTS
Header: TEST CREDIT
Header: REQUIRED COURSES FOR ADMISSION
Course Prefix: MATH 1A, Course Title: Calculus, Units: 5.00
Course Prefix: MATH 1B, Course Title: Calculus, Units: 5.00
Course Prefix: MATH 1AH, Course Title: Calculus - HONORS, Units: 5.00
Course Prefix: MATH 1BH, Course Title: Calculus - HONORS, Units: 5.00
Conjunction: AND
Conjunction: OR
Conjunction: AND
Attribute: Regular and honors courses may be combined to complete this series
Attribute: Regular and honors courses may be combined to complete this series
Course Prefix: MATH 1B, Course Title: Calculus, Units: 5.00
Course Prefix: MATH 1C, Course Title: Calculus, Units: 5.00
Course Prefix: MATH 1BH, Course Title: Calculus - HONORS, Units: 5.00
Course Prefix: MATH 1CH, Course Title: Calculus - HONORS, Units: 5.00
Conjunction: AND
Conjunction: OR
Conjunction: AND
Attribute: Regular and honors courses may be combined to complete